In [ ]:
import os
os.chdir('..')

In [ ]:
import inspect
import multiprocessing as mp
from typing import List, Tuple

import numba as nb
import numpy as np
import pandas as pd
import time

from neo_backtesting import Backtester
from strategy import boll_inbar
from util import read_candle_feather, transform_candle_np_struct, transform_np_struct
from joblib import Parallel, delayed

%load_ext line_profiler

In [ ]:
# 回测起始日
START_DATE = '20180301'
END_DATE = '20220614'

# 布林带周期与宽度
N = 100
B = 2

# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万
FACE_VALUE = 0.001  # 合约面值 0.001

COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 5

CONTRACT_TYPE = 'futures'  # 正向合约

# 模拟器参数
SIMULATOR_PARAMS = {
    'init_capital': INIT_CAPITAL, 
    'face_value': FACE_VALUE, 
    'comm_rate': COMM_RATE, 
    'liqui_rate': LIQUI_RATE, 
    'init_pos': 0
}

STRA = boll_inbar  # 要回测的策略

MIN_INTERVAL = '1m'  # 最小回测周期

ETH_PATHS = {
    '1m': '../binance_spot/ETH-USDT_1m.fea',
    '30m': '../binance_spot/ETH-USDT_30m.fea',
    '1h': '../binance_spot/ETH-USDT_1h.fea'
}

In [ ]:
backtester = Backtester(
    candle_paths=ETH_PATHS, 
    contract_type='futures', 
    simulator_params=SIMULATOR_PARAMS, 
    stra_module=boll_inbar)
factor_params = {
    'n': 100,
    'b': 2,
    'itl': '1h'
}
strategy_params = {
    'leverage': 1
}
%time results = backtester.run_detailed(
    start_date=START_DATE, 
    end_date=END_DATE, 
    init_capital=INIT_CAPITAL, 
    face_value=FACE_VALUE, 
    factor_params=factor_params, 
    strategy_params=strategy_params)

results['equity'] /= results['equity'].iat[0]
results

In [ ]:

fparams_list = STRA.get_default_factor_params_list()
sparams_list = STRA.get_default_strategy_params_list()

fparams_seqs = []

n = len(fparams_list)
n_cpus = os.cpu_count()

j = 0
for i in range(n_cpus):
    n_tasks = n // n_cpus
    if i < n % n_cpus:
        n_tasks += 1
    fparams_seqs.append(fparams_list[j : j + n_tasks])
    j += n_tasks

def search(fl):
    sparams_list = STRA.get_default_strategy_params_list()
    backtester = Backtester(
        candle_paths=ETH_PATHS, 
        contract_type='futures', 
        simulator_params=SIMULATOR_PARAMS, 
        stra_module=boll_inbar)
    t1 = time.time()
    df = backtester.run_gridsearch(
        start_date=START_DATE,
        end_date=END_DATE,
        init_capital=INIT_CAPITAL,
        face_value=FACE_VALUE,
        fparams_list=fl,
        sparams_list=sparams_list
    )
    return time.time() - t1, len(fl)

%time print(search(fparams_seqs[0]))